Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labeled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [4]:
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified .\notMNIST_large.tar.gz
Found and verified .\notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labeled A through J.

In [5]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

.\notMNIST_large already present - Skipping extraction of .\notMNIST_large.tar.gz.
['.\\notMNIST_large\\A', '.\\notMNIST_large\\B', '.\\notMNIST_large\\C', '.\\notMNIST_large\\D', '.\\notMNIST_large\\E', '.\\notMNIST_large\\F', '.\\notMNIST_large\\G', '.\\notMNIST_large\\H', '.\\notMNIST_large\\I', '.\\notMNIST_large\\J']
.\notMNIST_small already present - Skipping extraction of .\notMNIST_small.tar.gz.
['.\\notMNIST_small\\A', '.\\notMNIST_small\\B', '.\\notMNIST_small\\C', '.\\notMNIST_small\\D', '.\\notMNIST_small\\E', '.\\notMNIST_small\\F', '.\\notMNIST_small\\G', '.\\notMNIST_small\\H', '.\\notMNIST_small\\I', '.\\notMNIST_small\\J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [6]:
def show_random_image():
    for folder in train_folders:
        images = os.listdir(folder)
        print(folder.split("\\")[1])
        display(Image(filename=os.path.join(folder,images[0])))

show_random_image()

notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


notMNIST_large


Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [9]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

Pickling .\notMNIST_large\A.pickle.
.\notMNIST_large\A
Could not read: .\notMNIST_large\A\RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : cannot identify image file '.\\notMNIST_large\\A\\RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png' - it's ok, skipping.
Could not read: .\notMNIST_large\A\SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : cannot identify image file '.\\notMNIST_large\\A\\SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png' - it's ok, skipping.
Could not read: .\notMNIST_large\A\Um9tYW5hIEJvbGQucGZi.png : cannot identify image file '.\\notMNIST_large\\A\\Um9tYW5hIEJvbGQucGZi.png' - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.12825
Standard deviation: 0.443121
Pickling .\notMNIST_large\B.pickle.
.\notMNIST_large\B
Could not read: .\notMNIST_large\B\TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png : cannot identify image file '.\\notMNIST_large\\B\\TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png' - it's ok, skipping.
Full dataset tensor: (52911, 28, 28)
Mean: -0.00756303
Standard deviation: 0.

In [12]:
tmp_file = os.path.join('notMNIST_large\A', 'a2F6b28udHRm.png')
tmp_data = (ndimage.imread(tmp_file).astype(float) - pixel_depth / 2) / pixel_depth
print(tmp_data)

[[-0.5        -0.5        -0.5        -0.49607843 -0.5        -0.48823529
  -0.32745098 -0.06078431  0.19019608  0.35882353  0.44117647  0.49607843
   0.5         0.5         0.5         0.5         0.49607843  0.44117647
   0.3627451   0.19803922 -0.05294118 -0.31568627 -0.48431373 -0.5
  -0.49607843 -0.5        -0.5        -0.5       ]
 [-0.5        -0.5        -0.48431373 -0.5        -0.32745098  0.12352941
   0.44509804  0.5         0.49607843  0.5         0.5         0.5         0.5
   0.5         0.5         0.5         0.5         0.5         0.5
   0.49607843  0.5         0.44509804  0.14313725 -0.31176471 -0.5
  -0.48431373 -0.49607843 -0.5       ]
 [-0.49607843 -0.49607843 -0.49215686 -0.09607843  0.43333333  0.5
   0.49215686  0.48823529  0.49215686  0.49607843  0.5         0.5         0.5
   0.5         0.5         0.5         0.5         0.5         0.49607843
   0.49215686  0.48823529  0.48823529  0.5         0.44117647 -0.06862745
  -0.48431373 -0.5        -0.49607843]
 

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [15]:
with open('notMNIST_small\A.pickle', 'rb') as f:
    tmp_dataset = pickle.load(f)
    print(tmp_dataset[0,:])
    plt.imshow(tmp_dataset[0,:])

UnicodeDecodeError: 'ascii' codec can't decode byte 0xbf in position 3: ordinal not in range(128)

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [10]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Unable to process data from .\notMNIST_small\A.pickle : 'ascii' codec can't decode byte 0xbf in position 3: ordinal not in range(128)


UnicodeDecodeError: 'ascii' codec can't decode byte 0xbf in position 3: ordinal not in range(128)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [43]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

Finally, let's save the data for later reuse:

In [44]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [45]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800451


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [44]:

def overlap(dataset_a, dataset_b):
    overlaps = 0
    if dataset_a is not None and dataset_b is not None:
        for data_in_a in dataset_a[:]:
            for data_in_b in dataset_b[:]:
                if np.all(data_in_a == data_in_b):
                    overlaps += 1
    return overlaps

#print('%d overlaps between train dataset and validation dataset' % overlap(train_dataset, valid_dataset))

KeyboardInterrupt: 

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [47]:
from sklearn import datasets, linear_model

pickle_file = os.path.join(data_root, 'notMNIST.pickle')
try:
    f = open(pickle_file, 'rb')
    data_file = pickle.load(f)
    train_dataset = data_file['train_dataset']
    train_labels = data_file['train_labels']
    test_dataset = data_file['test_dataset']
    test_labels = data_file['test_labels']
except Exception as e:
    raise

#print(test_labels)

regr = linear_model.LinearRegression()
regr.fit(np.reshape(train_dataset, (train_dataset.shape[0], 784)), train_labels)

for index,val in enumerate(regr.predict(np.reshape(test_dataset, (test_dataset.shape[0], 784)))):
    print(val, test_labels[index])

0.165216134738 0
4.88823404364 6
8.6066327916 9
4.44401756406 6
3.48013657018 2
6.30966373233 7
6.44808338414 7
7.77056890401 9
6.78844427611 8
4.20400392925 5
2.10663466007 0
5.90374128237 8
3.18347216836 3
4.99045238449 9
3.05216233963 2
6.61169977062 3
6.39499511957 8
3.30316958656 3
6.12475055409 6
0.093099799869 1
3.56572070674 4
-0.236660615203 8
6.74129706585 3
6.73028782893 8
6.7161387616 9
4.39380577197 6
4.08653472154 4
4.29752801362 6
6.41056375236 4
4.49098265127 1
0.836146711314 2
2.99077579541 2
10.7271702623 9
3.89403353687 6
6.50492957827 7
6.43033662477 8
3.57730986047 5
3.33307889056 4
1.95879045328 1
6.02390185372 6
5.21824670002 6
6.15108299268 6
6.09942729596 7
-0.243091944076 0
2.08969528174 2
6.64794424612 6
6.26907295722 2
5.64391373956 1
6.05072493225 5
2.22046816347 0
1.80880461509 1
3.71693132014 4
3.76050945824 2
8.60532329302 9
5.42443608589 9
6.85672504509 8
3.81546541179 5
3.53230308054 4
4.75298447201 6
3.94101765247 2
6.85672504509 8
6.71482483055 8
4.7

7.59808842975 8
4.29723812493 9
3.54728286513 6
1.1795322552 1
5.83363921764 5
3.41685992085 2
3.96054613928 5
6.22314775706 7
9.77953613869 9
4.47523134985 5
6.55756489049 7
3.4782818186 2
5.06675627418 6
4.5439111092 4
3.66936431417 5
4.48887708769 6
2.02530947321 1
6.1505678587 7
4.49616618394 4
1.69149375042 3
3.2197478013 4
1.12804866013 1
4.57938934926 7
-0.363854722722 0
4.17754560301 9
8.83290496174 9
7.10026845877 8
4.19504149643 5
3.11654271887 3
5.42053494689 2
6.54440649092 8
5.15943437875 3
6.89821469025 7
8.31576951922 9
5.3905122015 8
4.84212153712 3
3.9811137258 2
4.51292651904 2
4.47497997653 6
1.53900210441 0
3.13556940329 2
4.67574755274 6
4.08061959206 6
6.96648613204 7
4.07544626423 4
1.09683080934 1
3.34677743651 4
0.902846084066 0
3.49206647958 5
1.5278977445 1
4.85152330563 5
5.57824668851 4
1.76851822357 2
3.80886215899 6
8.1076943473 4
7.44645704575 9
4.32169932074 5
9.505418134 9
7.81735407109 9
5.01396875825 6
6.13295233709 9
3.21519068757 3
0.991824883583 1

3.05895525336 5
1.68541417948 3
2.72156886914 5
6.12277923561 4
4.87740110124 4
2.52380691382 0
2.82682689598 2
4.11180966357 4
4.17210928517 5
4.95374488346 5
6.28723546275 7
7.00458992006 8
5.17347142537 5
3.67120494427 0
4.22780090071 6
1.55563175758 1
4.13642026673 5
6.58034576287 6
2.62173464144 2
5.04646031628 6
3.20029074925 2
5.40313385519 3
6.13894770154 8
2.89896312836 3
8.75082188633 9
6.84069412188 8
3.93677775677 0
3.85647686695 7
0.985846441207 0
4.96943491465 8
1.92322092116 2
1.34821245596 0
1.64010225607 1
4.65576069979 6
6.07401504597 8
2.3017275738 2
3.85299561001 2
3.76617177947 3
4.4856710398 0
7.88444278655 5
3.48176126004 3
0.96310740485 1
2.321550891 3
1.05784776567 0
5.49387253598 5
4.59763665971 1
0.0896938353409 0
4.80002332178 7
6.40358680161 3
2.31394803322 1
4.74562467789 6
4.44836507564 4
3.42122077178 5
5.9537063973 5
7.2791476923 7
2.18543676418 4
2.30341062339 0
4.06823077058 3
5.33308803844 6
4.40328545534 5
3.06157279422 6
2.50777673898 3
2.494876067

3.81739524068 4
3.35395419269 3
4.86874508007 8
0.257979642498 0
4.11440323514 0
4.28279519382 4
4.75290021654 5
2.53003052103 7
1.03412915135 1
2.99850168247 3
5.668967165 6
3.5831760163 3
3.54472997808 5
8.34473789319 9
6.35859117524 7
5.45794336323 8
3.1639621401 2
7.23717679672 6
1.05108722709 1
7.51615280502 8
9.5501431051 9
5.12255432819 5
4.35501758044 7
3.17906683283 2
0.0915547022316 0
3.992962985 2
3.31789710298 7
5.22536773102 9
11.1146310713 9
2.30318035995 1
2.05298712005 1
4.29950693698 4
6.85672504509 8
7.36325586262 9
2.23056806135 1
5.74402884451 7
8.83062424924 9
4.73413924158 0
6.952584432 4
3.83402573746 4
2.97456129106 6
4.05817120318 5
6.58312149272 8
8.36245949104 9
8.08498752763 9
0.867627969522 0
5.77799167958 7
7.47189840703 9
7.40779557885 6
2.61690492116 2
4.92969884491 3
1.1989303129 2
2.22810809326 2
5.7075584003 8
2.59429883064 4
3.10171690329 3
1.83813386666 2
4.05941207555 2
6.84680503906 8
4.38044873195 7
-0.31254486446 0
6.05923450834 5
7.13680620002 

3.5459459458 2
7.5770316417 8
5.06195211972 6
1.42687862452 1
4.74796701542 2
2.58728830493 3
1.82356221602 2
6.89662281756 9
1.31524351833 1
2.77271131714 1
4.89526293675 5
6.66729350647 7
1.44454478066 1
6.37978149837 7
2.00177413807 1
4.1723480727 1
8.91067681444 9
5.93205690397 4
4.4166560099 5
1.0850979299 0
7.06389244786 8
7.4556845817 8
6.98835589452 9
5.60373054983 4
3.16843901605 1
5.28072262782 0
3.39050845134 1
7.15471868107 8
6.95398643854 7
2.7568366214 2
8.49923872385 8
0.0160921001123 0
7.33676553679 6
4.92963108594 7
4.80105914429 4
2.93179482788 3
3.55675028229 3
4.73594379124 6
2.10459696705 1
4.07477035496 4
7.24764421426 8
4.39298802643 1
2.64819684894 5
5.91992210935 7
4.75287952083 5
7.85992872399 9
5.42415840374 4
4.15034462756 4
4.70888950126 4
8.15107080562 9
6.59581635798 7
3.24462805885 3
3.09998760892 3
10.1777147743 9
3.54922589308 4
4.58288647361 7
4.34034821095 8
1.6645721461 1
0.764108849268 1
6.85672504509 8
1.82263099682 2
7.15852589914 8
10.0566803275

...

3.86501971589 4
4.86712497269 2
2.6472042282 2
3.14929952583 3
3.10372853799 2
1.75240827545 1
3.73501750299 0
1.17875229075 1
7.79560238988 9
6.46156607825 9
5.96253737814 5
6.35471520325 8
6.71649987186 8
1.09462837144 0
3.51573391757 5
6.02680090226 5
1.80940644587 2
6.69972033697 7
1.5581319827 1
0.920568870799 0
6.32151792717 7
1.22729803373 1
3.66698208751 4
3.54322697811 4
3.28480119264 3
0.114924442888 7
4.87332741576 8
7.81748579522 9
2.78874938182 3
6.59193304698 9
2.27816157593 1
7.54144298056 9
3.2695938996 5
7.51667785901 8
2.96225703787 3
4.76125617649 9
1.29712871647 2
0.763966224699 1
3.26976152276 3
3.66505976197 2
4.33721059886 4
3.58026845776 4
2.83836777035 2
1.52385731013 1
6.10299168879 5
6.86655116942 7
3.94898827441 5
5.26565233195 4
0.864275737533 2
2.33718972296 3
4.01223322888 4
3.75788303519 1
5.89696616934 8
3.55881281514 4
2.9787866642 8
6.57213936181 5
2.00717721656 2
8.49229877757 9
1.80998986382 0
1.46490413519 0
2.80562482101 0
3.54089438813 4
1.450478

7.31525717675 6
6.29290324724 2
0.824603978151 0
1.38741909088 1
7.05005576696 8
5.96567105148 5
6.44513580323 7
1.83871526221 1
5.04276379427 5
7.34144487641 8
7.06219507684 8
4.9347822449 8
5.13241163741 6
3.5595028696 5
4.40541162722 6
3.07551123414 2
1.89590018083 0
10.4818244079 9
7.28701199938 5
9.18491796224 9
7.09609859711 9
4.51435654501 8
5.90906076447 7
1.46336919759 1
4.91752390061 1
1.19794497606 2
6.22913826963 4
1.36980914486 1
2.77482567812 6
2.46073401548 1
5.32757351753 5
7.05752203429 8
4.68641156126 4
7.89322592854 9
6.53405955198 8
1.13374174215 1
4.25211403987 4
3.14978920325 3
5.45810523921 7
4.43583093451 8
4.86055474484 4
5.57630423958 5
6.96120178096 0
0.245191705418 0
3.11212034012 3
3.23486492419 4
4.18866785243 6
0.877369251477 1
5.21574231776 7
3.55447221427 2
4.87679042118 6
5.7965228534 5
4.26469442923 4
2.23166068681 1
3.04075230075 3
5.98819297029 7
5.44278647855 6
4.8542838488 4
3.75952762977 4
2.20683701284 1
9.30876045863 9
2.44189719966 3
4.0456234

0.213150937066 0
6.16898524135 6
3.85221475484 3
2.25187502948 2
3.03453494133 1
3.74265190598 3
7.83196392475 9
4.85519359749 6
2.50656673821 1
5.1906243156 5
2.49965856449 3
2.42280675366 1
4.14945037737 5
5.04263689562 7
3.62721614054 0
3.13172872356 4
0.126293670945 0
5.75320010206 9
2.54839037039 2
1.0830860733 1
4.12312260016 4
0.247290454226 1
4.68865442344 7
2.10248414971 2
2.29514719863 1
6.44435493927 7
2.19404185911 1
3.51126900741 5
3.45083798903 2
4.95361138519 4
0.733727570127 0
2.87663845412 2
3.36536887715 3
2.17422841497 6
4.5846012076 5
3.20508797673 1
7.93765478356 9
3.09384540756 3
1.08244725456 2
5.29044919464 7
2.94560713406 5
3.47204982998 0
5.4674906823 6
4.24120455743 4
7.7036240281 9
2.02094750117 3
2.53185448223 0
6.3543986117 9
6.43926502967 8
5.33317293467 7
2.81767755072 3
1.92944519494 1
5.62670262012 4
8.22991430116 9
4.91152511109 6
-0.315040843807 0
4.28160947245 5
2.52747074009 2
3.33555184754 1
3.91596086412 4
4.04769808806 6
5.05597160724 4
10.04565

3.34983696367 6
4.23477638731 6
9.25625277362 9
3.7654026314 6
3.78793597111 3
2.70183512226 3
2.16333735472 2
2.65224852129 1
0.148342591161 0
2.94342203354 2
4.09217090638 3
4.94543963561 4
4.17907640324 7
3.77936305314 8
3.51334907032 2
6.53982025571 7
5.15932080984 5
4.02302019162 6
4.08296095902 5
1.27038467504 2
6.59916632119 9
2.61744544433 1
6.32698044804 5
2.20239773786 5
2.28712247097 3
2.82622064878 2
6.23820419859 7
3.35928882417 3
2.39788334771 2
3.2723392707 3
7.3252202656 8
1.20975704234 1
5.95791609424 5
2.20809706595 2
7.54067914024 8
5.52566913352 5
1.35243996006 0
2.77806362811 4
4.11809590115 2
3.81965221221 5
5.4020875054 7
3.5141517522 0
-0.665422774571 0
7.30368339637 6
3.07937034015 3
8.01030394648 8
1.63973571661 2
3.27510642552 0
4.63364220447 7
2.59166144924 1
5.73231836539 6
3.3899409543 0
6.69047042745 8
7.36504535252 8
5.08985082722 2
4.52711480618 5
4.16576822204 7
3.72649006063 0
3.19818569507 3
3.98730134975 1
3.85832907675 2
4.48537671981 4
3.588216234

6.38757797203 7
1.41444962229 4
6.78495766977 9
2.45772413514 1
4.84041285586 4
3.64590936375 1
6.75733517348 7
2.21410779326 2
1.42971352759 1
8.04884885981 8
1.71308162928 1
2.99159431211 3
5.06840549054 2
3.4678011996 4
1.91945148377 1
9.03487880177 9
2.24026476788 5
-0.177644096031 0
-1.83103114352 0
6.03716856975 7
6.86246152955 8
5.2278576893 6
5.14708647477 5
2.62426778643 2
4.0672157722 8
2.96035142406 3
5.16258339866 6
-0.102603922511 1
5.06050320283 5
5.08187296076 6
5.11858649278 6
4.57956663904 4
3.51242239406 2
2.14374542178 2
6.17024873196 8
8.46865886281 9
2.60333101393 3
-0.127962542091 0
6.17113691019 8
4.5797767656 1
2.96779308057 4
11.5336482269 9
3.50292396722 2
1.92280761694 1
3.42566901739 3
3.73807132825 3
5.82207268147 7
9.52474939294 9
7.46208561761 8
5.76687723408 9
3.26887648368 6
5.85023064119 6
5.52685533446 7
4.34909554046 9
1.98002113805 0
9.64412823455 9
6.611394905 9
4.15790452258 6
4.65517692645 3
5.21998751938 5
5.56702725159 2
5.54331701994 5
3.03743